# **Solving the LP:**

In [ ]:
!pip install pyomo
!apt-get install -y glpk-utils

from pyomo.environ import *


model = ConcreteModel()


plants = [1, 2, 3]
distribution_centers = list(range(1, 13))

# Data
capacity = {1: 120, 2: 90, 3: 100}
production_cost = {1: 8, 2: 7, 3: 9}
demand = {1: 20, 2: 30, 3: 40, 4: 35, 5: 15, 6: 19, 7: 25, 8: 20, 9: 18, 10: 12, 11: 30, 12: 40}
selling_price = 20

transport_cost = {
    (1, 1): 2, (1, 2): 3, (1, 3): 2, (1, 4): 3, (1, 5): 5, (1, 6): 4.4, (1, 7): 3.2, (1, 8): 3, (1, 9): 1.5, (1, 10): 3, (1, 11): 6.0, (1, 12): 1.0,
    (2, 1): 1.9, (2, 2): 3, (2, 3): 2.4, (2, 4): 2.4, (2, 5): 2, (2, 6): 4.1, (2, 7): 2.8, (2, 8): 3, (2, 9): 1.7, (2, 10): 2, (2, 11): 1.0, (2, 12): 4.5,
    (3, 1): 3.1, (3, 2): 1.8, (3, 3): 2.8, (3, 4): 3.4, (3, 5): 2, (3, 6): 2.9, (3, 7): 2.2, (3, 8): 3.2, (3, 9): 2, (3, 10): 2.8, (3, 11): 4.5, (3, 12): 6.0
}

# Decision Variables
model.x = Var(plants, distribution_centers, domain=NonNegativeReals)

# Objective Function
def objective_rule(model):
    return sum((selling_price - transport_cost[i, j]) * model.x[i, j] for i in plants for j in distribution_centers) - \
           sum(production_cost[i] * sum(model.x[i, j] for j in distribution_centers) for i in plants)

model.objective = Objective(rule=objective_rule, sense=maximize)

# Supply Constraints
model.supply_constraints = ConstraintList()
for i in plants:
    model.supply_constraints.add(sum(model.x[i, j] for j in distribution_centers) <= capacity[i])

# Demand Constraints
model.demand_constraints = ConstraintList()
for j in distribution_centers:
    model.demand_constraints.add(sum(model.x[i, j] for i in plants) == demand[j])


solver = SolverFactory('glpk',executable='/usr/bin/glpsol')
solver.solve(model, tee=True)


import pandas as pd


data = []
for i in plants:
    for j in distribution_centers:
        if model.x[i, j].value > 0:
            data.append([i, j, model.x[i, j].value])


df = pd.DataFrame(data, columns=["Plant", "Distribution Center", "Units Shipped"])

# table
print(df.to_string(index=False))

# total profit
print(f"\nTotal Profit: {model.objective():.2f}")


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev
The following NEW packages will be installed:
  glpk-utils libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
0 upgraded, 5 newly installed, 0 to remove and 20 not upgraded.
Need to get 625 kB of archives.
After this operation, 2,158 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libsuitesparseconfig5 amd64 1:5.10.1+dfsg-4build1 [10.4 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libamd2 amd64 1:5.10.1+dfsg-4build1 [21.6 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 libcolamd2 amd64 1:5.10.1+dfsg-4build1 [18.0 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libglpk40 amd64 5.0-1 [361 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy/universe amd64 glpk-ut

**Maximum profit: 3065.20**